In [9]:
import torch
import pandas as pd

#data format setup:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
input_data = torch.tensor([0,0,0,0,0,0,0,0,0], dtype=torch.float)

def play_ground(h, w, agent_a, agent_b, agent_c, agent_d, ball):
    rows = h
    columns = w
    init_ground = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.float)
    return init_ground




def agent_a(perception, state):
    action = 0
    state = 0
    return (action, state)



if __name__ == "__main__":
    print(play_ground(12,9,[-1, 1, -1, 3],[-1, 4, -1, 6], [0, 6, 0, 4], [0, 3, 0, 1], [3, 3]))


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]])


[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]